In [1]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/en2sparql

Mounted at /content/drive
/content/drive/MyDrive/en2sparql


In [3]:
import torch
import json
from transformers import pipeline, AutoTokenizer, GPT2LMHeadModel

gpt2_model = GPT2LMHeadModel.from_pretrained("./ep10")
tokenizer = AutoTokenizer.from_pretrained("./ep10")
gpt2 = pipeline('text-generation', model=gpt2_model, max_new_tokens=400, return_full_text=False,tokenizer = tokenizer, device=0)

In [4]:

def divide_chunks(l_, n_):
    for i_ in range(0, len(l_), n_):
        yield l_[i_:i_ + n_]


def load_json(file__name):
    data_file = open(file__name, "r", encoding='utf-8')
    file_data = json.loads(data_file.read())
    data_file.close()
    return file_data

def save_json(filename, data):
    with open(filename, "w", encoding="utf-8") as json_file:
        print(json.dumps(data), file=json_file)


def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st

def main(filename, batch=103):
    data = load_json(filename)
    query_list = data.get("questions")
    suggestions = data.get("suggestions")
    sparql = data.get("sparql")
    gs = data.get("generated_sparql")
    print(len(query_list))

    q_list = query_list[len(gs):]  # list(divide_chunks(query_list, n))

    if data.get("prompt_len") is None:
        lens = []
        for i, question in enumerate(q_list):
            res_ = tokenizer.encode(question)
            len_ = len(res_)
            lens.append(len_)
            if len_ > 600:
                q_list[i] = tokenizer.decode(res_[-600:])
                print(q_list[i])
    else:
        lens = data.get("prompt_len")

    result = {"orig_questions": query_list, "sparql": sparql, "generated_sparql": gs, "prompt_len": lens,
              "questions": q_list,
              "suggestions": suggestions}
    save_json(filename, result)

    g_list = list(divide_chunks(q_list, batch))

    for questions in g_list:
        print(len(gs))
        res = gpt2(questions)

        gst = [x[0]["generated_text"] for x in res]
        gs += gst
        result = {"orig_questions": query_list, "sparql": sparql, "generated_sparql": gs, "prompt_len": lens,
                  "questions": q_list,
                  "suggestions": suggestions}
        save_json(filename[:-5]+"_tok.json", result)



In [9]:
main("pt_gpt2-large_cheat_3_shots.json")

513
model_lbl. } ORDER BY DESC(?value) LIMIT 1 } } 
 input (English text): What is the best performing model benchmarking the Atari 2600 James Bond dataset in terms of Medium Human-Normalized Score metric?
 output (Sparql query): SELECT DISTINCT?model?model_lbl WHERE {?metric a orkgc:Metric; rdfs:label?metric_lbl. FILTER (str(?metric_lbl) = "Medium Human-Normalized Score") { SELECT?model?model_lbl WHERE {?dataset a orkgc:Dataset; rdfs:label?dataset_lbl. FILTER (str(?dataset_lbl) = "Atari 2600 James Bond")?benchmark orkgp:HAS_DATASET?dataset; orkgp:HAS_EVALUATION?eval.?eval orkgp:HAS_VALUE?value; orkgp:HAS_METRIC?metric.?cont orkgp:HAS_BENCHMARK?benchmark; orkgp:HAS_MODEL?model.?model rdfs:label?model_lbl. } ORDER BY DESC(?value) LIMIT 1 } } 
 input (English text): Indicate the model that performed best in terms of Bits per byte metric on the Classical music, 5 seconds at 12 kHz benchmark dataset?
 output (Sparql query): SELECT DISTINCT?model?model_lbl WHERE {?metric a orkgc:Metric; rdf

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

KeyboardInterrupt: ignored